In [107]:
import math
import json
import datetime
import pandas as pd
from jira import JIRA
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery

table_suffix = (datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1)).strftime("%Y%m%d")
check_date = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=45)
check_date_string = (datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=45)).strftime("%Y-%m-%d")

jiraOptions = {'server': "server"}
jira = JIRA(options=jiraOptions, basic_auth=(
    "name", "token"))

project_name = 'RECRUITMENT'
issues = jira.search_issues('project = "{}"'.format(project_name),0,1)
number_issue = int(issues[0].key.split("-")[1])
number_page = math.ceil(number_issue/100)

all_data = []
for i in range(number_page):
    all_issue = jira.search_issues('project = "{}" and created > "{}"'.format(project_name, check_date_string),i*100,100)

    for singleIssue in all_issue:
        all_data.append([singleIssue.key, singleIssue.fields.summary, singleIssue.fields.reporter.displayName, singleIssue.fields.status.name, singleIssue.fields.created])
        
df = pd.DataFrame(all_data)
df.columns = ["key", "name", "reporter", "status", "created_at"]
df = df.set_index("key")
df["created_at"] = pd.to_datetime(df.created_at.str.slice(0,10))
df = df[(df.created_at > check_date)]

for key in df.index:
    changelog = jira.issue(key, expand='changelog').changelog

    for history in changelog.histories:
        for item in history.items:
            if item.field == 'status':
                all_change_log.append([key, history.created, item.fromString, item.toString])
                
df1 = pd.DataFrame(all_change_log)
df1.columns = ["key", "updated_at", "from", "to"]
df1 = df1.set_index("key")

df2 = df.join(df1)
df2["updated_at"] = pd.to_datetime(df2.updated_at.str.slice(0,10))
df2 = df2.reset_index()

data = []
for i in range(len(df2)):
    data.append(json.loads(df2.iloc[i, :].to_json()))

#find schema
schema = []
new_col_name = []
for name in data[0].keys():
    new_col_name.append("_".join(name.split(" ")))
    schema.append(bigquery.SchemaField("_".join(name.split(" ")), bigquery.enums.SqlTypeNames.STRING))

#connect to bigquery
client = bigquery.Client.from_service_account_json('google_token.json')
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
)
table_id = "taptap-c2c5f.manual_dwh.jira_hr_" + table_suffix
job = client.load_table_from_json(
    data, table_id, job_config=job_config
)
print(job.result())

LoadJob<project=taptap-c2c5f, location=US, id=3c7e45f8-53ff-4ab4-afeb-a7499658140e>
